# **Imports**

In [ ]:
!pip install tokenizers
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 31.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 13.0 MB/s 
     |████████████████████████████████| 120 kB 52.0 MB/s 
     |████████████████████████████████| 212 kB 60.1 MB/s 
     |████████████████████████████████| 115 kB 56.2 MB/s 
     |████████████████████████████████| 127 kB 52.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 9.4 MB/s 


In [ ]:
import os
import gzip
import json
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import plotly.express as ex
from pathlib import Path
from sklearn.model_selection import train_test_split
from tokenizers.implementations.byte_level_bpe import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from datasets import load_dataset
from torch.utils.data import Dataset,DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import RobertaForSequenceClassification
from transformers.data.metrics import simple_accuracy
from sklearn.metrics import f1_score

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Data Setting**

### Data Collection

**Constants**

In [ ]:
Size = 30000

In [ ]:
LANGUAGES = {
    "GO": 0,
    "Java": 1,
    "JavaScript": 2,
    "PHP": 3,
    "Python": 4,
    "Ruby": 5,
    "HTML": 6,
    "SQL": 7,
    "C": 8,
    "Rust": 9,
}

In [ ]:
Languages = ['GO','Java','JavaScript','PHP','Python','Ruby','HTML','SQL','C','Rust']

**Data Extraction**

In [ ]:
DataFrame = pd.DataFrame(columns = ['code', 'language'])
for l in Languages:
  dataset = load_dataset("codeparrot/github-code", streaming=True, split='train', languages = [l])
  dataset = dataset.remove_columns(['repo_name', 'path', 'license', 'size'])
  dataset = dataset.take(Size)
  DataFrame = DataFrame.append(pd.DataFrame(dataset))
DataFrame.to_parquet('/content/gdrive/MyDrive/MLA Project/10Languages.parquet')

**Data** **Visulaization**

In [ ]:
dataset_count = DataFrame.groupby('language').count()
ex.pie(dataset_count.index[:],names='language',title='Proportion Of Different Languages',hole=0.33)

### Reading Data

In [ ]:
DataFrame = pd.read_parquet('/content/gdrive/MyDrive/MLA Project/10LanguagesbalancedData.parquet')

In [ ]:
Data = DataFrame['code'].to_numpy()
Label = DataFrame['language'].to_numpy()

###Splitting Data

In [ ]:
train_data, test_data , train_labels, test_labels = train_test_split(Data, Label, test_size=0.3, random_state=0,stratify=Label)
validation_data, test_data, validation_labels, test_labels=train_test_split(test_data, test_labels, test_size=(2/3), random_state=0, stratify=test_labels)

In [ ]:
print(len(train_data))
print(len(validation_data))
print(len(test_data))

196154
28022
56044


### Saving Data

In [ ]:
np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/trainData.npy', train_data)
np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/trainLabels.npy', train_labels)
np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/validationData.npy', validation_data)
np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/validationLabels.npy', validation_labels)
np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/testData.npy', test_data)
np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/testLabels.npy', test_labels)

# **Removing** **Comments**

### Script

In [ ]:
DataFrame = pd.read_parquet('/content/gdrive/MyDrive/MLA Project/10Languages.parquet')

In [ ]:
LANGUAGES = {
    "Assembly": [';','\n'],
    "Batchfile": ['::','\n','REM','\n'],
    "C": ['/*','*/','//','\n'],
    "C#": ['/*','*/','//','\n'],
    "C++": ['/*','*/','//','\n'],
    "CMake": ['#[[',']]','#','\n'],            
    "CSS": ['/*','*/'],
    "Dockerfile": ['#','\n'],
    "FORTRAN": ['!','\n'],
    "GO": ['/*','*/','//','\n'],
    "Haskell": ['--','\n','{-','-}'],
    "HTML": ['<!--', '-->'],
    "Java": ['/*','*/','//','\n'],
    "JavaScript": ['/*','*/','//','\n'],
    "Julia": ['#','\n','#=','=#'],
    "Lua": ['--[[','--]]','--','\n'], 
    "Markdown": ['[',']: #','[//]: # (',')','[//]: # "','"','[]: # (',')','[]: # "','"','<!--','-->'],
    "Makefile": ['#','\n'],
    "PHP": ['/*','*/','//','\n'],
    "Perl": ['#','\n','=begin','=end'],
    "PowerShell": ['#','\n','<#','#>'],
    "Python": ['#','\n'],
    "Ruby": ['#','\n','=begin','=end'],
    "Rust": ['/*','*/','//','\n'],
    "SQL": ['/*','*/','--','\n'],
    "Scala": ['/*','*/','//','\n'],
    "Shell": [": '", " '"],
    "TypeScript": ['//','\n','/*','*/'],
    "TeX": ['%','\n'],
    "Visual Basic": ["'","\n"]
}

In [ ]:
def removeSpecialComments(code, specialChars, shell=False):
  lines = code.splitlines()
  temp = []
  for line in lines:
    if shell:
      tempLine = line.strip()
    else:
      tempLine = line
    if tempLine == '':
      continue
    if tempLine[0] in specialChars:
      temp.append(line)
  for line in temp:
    lines.remove(line)
  code = "\n".join(lines)
  return code

In [ ]:
def RemoveComments(code, language):
  identifier = LANGUAGES[language]
  for k in range(0, len(identifier), 2):
    start = identifier[k]
    end = identifier[k+1]
    temp = []
    commentIndex = []
    for i in range(len(code)):
      if code[i : i + len(start)] == start:
        for j in range(i+len(start)+1, len(code)):
          if code[j : j + len(end)] == end and code[j - 1 : j - 1 + len(start)] != start:
            commentIndex.append((i, j+len(end)))
            break
    temp[:0] = code
    for l,m in reversed(commentIndex):
      del temp[l:m + len(end)]
    code = ''.join(temp)
  if language == "FORTRAN":
    code = removeSpecialComments(code, ['*', 'C', 'c', 'd', 'D'])
  elif language == 'Shell':
    code = removeSpecialComments(code, ['#'], True)
  return code

In [ ]:
new_df = pd.DataFrame(columns = ['code', 'language'])
languages=DataFrame.language.unique()
for l in range(len(languages)):
  for i in range(int(len(DataFrame)/len(languages))):
    x = DataFrame.loc[DataFrame['language'] == languages[l]]["code"][i]
    y = RemoveComments(x, languages[l])
    new_row = {'code':y, 'language':languages[l]}
    new_df = new_df.append(new_row, ignore_index=True)

new_df.to_parquet('/content/gdrive/MyDrive/MLA Project/10LanguagesWithoutComments.parquet')

### Balancing Data

In [ ]:
DataFrame = pd.read_parquet('/content/gdrive/MyDrive/MLA Project/10LanguagesWithoutComments.parquet')

In [ ]:
DataFrame.groupby('language').count()

,code
language,
C,30000
GO,30000
HTML,30000
Java,30000
JavaScript,30000
PHP,30000
Python,30000
Ruby,30000
Rust,30000


In [ ]:
languages=DataFrame.language.unique()
new_df = pd.DataFrame(columns = ['code', 'language'])
for l in range(len(languages)):
  for i in range(int(len(DataFrame)/len(languages))):
    x = DataFrame['code'][(l*(int(len(DataFrame)/len(languages))))+i]
    if len(x) != 0 :
      new_row = {'code':x, 'language':languages[l]}
      new_df = new_df.append(new_row, ignore_index=True)

In [ ]:
new_df.groupby('language').count()

,code
language,
C,29497
GO,29686
HTML,29989
Java,29928
JavaScript,29781
PHP,29999
Python,28931
Ruby,29440
Rust,28022


In [ ]:
new_df.to_parquet('/content/gdrive/MyDrive/MLA Project/10LanguagesWithoutEmptyFiles.parquet')

In [ ]:
new_df = pd.read_parquet('/content/gdrive/MyDrive/MLA Project/10LanguagesWithoutEmptyFiles.parquet')

In [ ]:
len(new_df)

294192

In [ ]:
files_count_map = {
    "C": 29497,
    "GO": 29686,
    "HTML":29989,
    "Java": 29928,
    "JavaScript": 29781,
    "PHP": 29999,
    "Python": 28931,
    "Ruby": 29440,
    "Rust": 28022,
    "SQL": 28919,
    }

In [ ]:
# Mapping
languages=new_df.language.unique()
files_count = []
for i in range (len(languages)):
  files_count.append(files_count_map[languages[i]])
files_count = np.array(files_count)

# Making new df with balanced data
final_df = pd.DataFrame(columns = ['code', 'language'])
start = 0
end = 0
for l in range (len(languages)):
  end +=files_count[l]
  count = 0
  for j in range(start,end):
    if count < 28022 :
      x = new_df['code'][j]
      new_row = {'code':x, 'language':languages[l]}
      final_df = final_df.append(new_row, ignore_index=True)
      count +=1
  start +=files_count[l]

In [ ]:
final_df.groupby('language').count()

,code
language,
C,28022
GO,28022
HTML,28022
Java,28022
JavaScript,28022
PHP,28022
Python,28022
Ruby,28022
Rust,28022


In [ ]:
dataset_count = final_df.groupby('language').count()
ex.pie(dataset_count.index[:],names='language',title='Proportion Of Different Languages',hole=0.33)

In [ ]:
final_df.to_parquet('/content/gdrive/MyDrive/MLA Project/10LanguagesbalancedData.parquet')

# **Tokenizer**

### Load data

In [ ]:
trainData = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/trainData.npy', allow_pickle=True)
trainLabels = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/trainLabels.npy', allow_pickle=True)
testData = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/testData.npy', allow_pickle=True)
testLabels = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/testLabels.npy', allow_pickle=True)
validationData = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/validationData.npy', allow_pickle=True)
validationLabels = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/validationLabels.npy', allow_pickle=True)

KeyboardInterrupt: ignored

### Training

In [ ]:
text = []
file_count = 1
for i in tqdm(range(1, trainData.size + 1)):
  text.append(trainData[i-1])
  if i % 19600 == 0:
    with open(f'/content/gdrive/MyDrive/MLA Project/dump/{file_count}.txt', 'w') as fp:
      fp.write('\n'.join(text))
      file_count += 1
      text = []
with open(f'/content/gdrive/MyDrive/MLA Project/dump/{file_count}.txt', 'w') as fp:
      fp.write('\n'.join(text))

100%|██████████| 196154/196154 [00:09<00:00, 21106.70it/s]


In [ ]:
from pathlib import Path
paths = [str(x) for x in Path('/content/gdrive/MyDrive/MLA Project/dump').glob('**/*.txt')]

In [ ]:
print(paths)

['/content/gdrive/MyDrive/MLA Project/dump/1.txt', '/content/gdrive/MyDrive/MLA Project/dump/2.txt', '/content/gdrive/MyDrive/MLA Project/dump/3.txt', '/content/gdrive/MyDrive/MLA Project/dump/4.txt', '/content/gdrive/MyDrive/MLA Project/dump/5.txt', '/content/gdrive/MyDrive/MLA Project/dump/6.txt', '/content/gdrive/MyDrive/MLA Project/dump/7.txt', '/content/gdrive/MyDrive/MLA Project/dump/8.txt', '/content/gdrive/MyDrive/MLA Project/dump/9.txt', '/content/gdrive/MyDrive/MLA Project/dump/10.txt', '/content/gdrive/MyDrive/MLA Project/dump/11.txt']


In [ ]:
from tokenizers import ByteLevelBPETokenizer
# initialize
tokenizer = ByteLevelBPETokenizer()
# and train
tokenizer.train(files=paths, vocab_size=50000 , min_frequency=2,
                special_tokens=['<|endoftext|>', '<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [ ]:
tokenizer.save_model('/content/gdrive/MyDrive/MLA Project/10LangaugesTokenizer')

['/content/gdrive/MyDrive/MLA Project/10LangaugesTokenizer/vocab.json',
 '/content/gdrive/MyDrive/MLA Project/10LangaugesTokenizer/merges.txt']

### Tokenizing Data

In [ ]:
# Set up tokenizer
tokenizer = ByteLevelBPETokenizer("/content/gdrive/MyDrive/MLA Project/10LanguagesTokenizer/vocab.json", "/content/gdrive/MyDrive/MLA Project/10LanguagesTokenizer/merges.txt",)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")), ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)
tokenizer.enable_padding(length=512)

In [ ]:
for i in tqdm(range(19)):
  encoding = tokenizer.encode_batch(trainData[10000*i:10000*(i+1)])
  en_train_data = [e.ids for e in encoding]
  np.save(f'/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtrainData{i}.npy', en_train_data)
i += 1
encoding = tokenizer.encode_batch(trainData[10000*i:])
en_train_data = [e.ids for e in encoding]
np.save(f'/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtrainData{i}.npy', en_train_data)

100%|██████████| 19/19 [17:38<00:00, 55.72s/it]


In [ ]:
temp = []
for i in tqdm(range(20)):
  temp.extend(np.load(f'/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtrainData{i}.npy'))
np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtrainData.npy', temp)

100%|██████████| 20/20 [00:04<00:00,  4.54it/s]


In [ ]:
for i in tqdm(range(2)):
  encoding = tokenizer.encode_batch(validationData[10000*i:10000*(i+1)])
  en_validation_data = [e.ids for e in encoding]
  np.save(f'/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDvalidationData{i}.npy', en_validation_data)
i += 1
encoding = tokenizer.encode_batch(validationData[10000*i:])
en_validation_data = [e.ids for e in encoding]
np.save(f'/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDvalidationData{i}.npy', en_validation_data)

100%|██████████| 2/2 [01:57<00:00, 58.97s/it]


In [ ]:
temp = []
for i in tqdm(range(3)):
  temp.extend(np.load(f'/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDvalidationData{i}.npy'))
np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDvalidationData.npy', temp)

100%|██████████| 3/3 [00:00<00:00, 12.95it/s]


In [ ]:
del validationData
del en_validation_data

In [ ]:
for i in tqdm(range(5)):
  encoding = tokenizer.encode_batch(testData[10000*i:10000*(i+1)])
  en_test_data = [e.ids for e in encoding]
  np.save(f'/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtestData{i}.npy', en_test_data)
i += 1
encoding = tokenizer.encode_batch(testData[10000*i:])
en_test_data = [e.ids for e in encoding]
np.save(f'/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtestData{i}.npy', en_test_data)

100%|██████████| 5/5 [04:49<00:00, 57.92s/it]


In [ ]:
temp = []
for i in tqdm(range(6)):
  temp.extend(np.load(f'/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtestData{i}.npy'))
np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtestData.npy', temp)

100%|██████████| 6/6 [00:00<00:00, 10.73it/s]


In [ ]:
del testData
del en_test_data

# **Transformer**

### Loading Data

In [ ]:
trainData = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtrainData.npy')
validationData = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDvalidationData.npy')
testData = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/TOKENIZEDtestData.npy')
trainLabels = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/trainLabels.npy', allow_pickle=True)
testLabels = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/testLabels.npy', allow_pickle=True)
validationLabels = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesBalancedData/validationLabels.npy', allow_pickle=True)

### Loading Model

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('/content/gdrive/MyDrive/MLA Project/10LanguagesModel/model')

## Training Script

**Constants**

In [ ]:
CODEBERTA_PRETRAINED = "huggingface/CodeBERTa-language-id"
EVALUATE = False
LANGUAGES = {
    "GO": 0,
    "Java": 1,
    "JavaScript": 2,
    "PHP": 3,
    "Python": 4,
    "Ruby": 5,
    "HTML": 6,
    "SQL": 7,
    "C": 8,
    "Rust": 9,
}

**Data Class**

In [ ]:
class CodeDataset(Dataset):
    def __init__(self, data, label, language):
      self.data = data
      self.label = label
      self.language = language

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
      return torch.tensor(self.data[index]), torch.tensor(self.language[self.label[index]])

In [ ]:
train_dataset = CodeDataset(trainData, trainLabels, LANGUAGES)
eval_dataset = CodeDataset(validationData, validationLabels, LANGUAGES)
test_dataset = CodeDataset(testData, testLabels, LANGUAGES)

**Model**

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(CODEBERTA_PRETRAINED, num_labels=len(LANGUAGES), ignore_mismatched_sizes=True)

**DataLoader**

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

**Miscellaneous**

In [ ]:
model.to("cuda")
model.train()
for param in model.roberta.parameters():
    param.requires_grad = False
## ^^ Only train final layer.

In [ ]:
print(f"num params:", model.num_parameters())
print(f"num trainable params:", model.num_parameters(only_trainable=True))

num params: 83473950
num trainable params: 613662


**Validation & Test**

In [ ]:
def evaluate(test = False):
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = np.empty((0), dtype=np.int64)
    out_label_ids = np.empty((0), dtype=np.int64)

    model.eval()
    if test == False :
      dataloader = DataLoader(eval_dataset, batch_size=128)
    else:
      dataloader = DataLoader(test_dataset, batch_size=128)
    for step, (input_ids, labels) in enumerate(tqdm(dataloader, desc="Eval")):
        with torch.no_grad():
            outputs = model(input_ids=input_ids.to("cuda"), labels=labels.to("cuda"))
            loss = outputs[0]
            logits = outputs[1]
            eval_loss += loss.mean().item()
            nb_eval_steps += 1
        preds = np.append(preds, logits.argmax(dim=1).detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, labels.detach().cpu().numpy(), axis=0)
        del input_ids
        del labels
        del outputs
        torch.cuda.empty_cache()
    eval_loss = eval_loss / nb_eval_steps
    acc = simple_accuracy(preds, out_label_ids)
    f1 = f1_score(y_true=out_label_ids, y_pred=preds, average="macro")
    if test == False :
      print("=== Eval: loss ===", eval_loss)
      print("=== Eval: acc. ===", acc)
      print("=== Eval: f1 ===", f1)
    else:
      print("=== Test: loss ===", eval_loss)
      print("=== Test: acc. ===", acc)
      print("=== Test: f1 ===", f1)
    return (eval_loss, acc)

**Training**

In [ ]:
trainLA = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesModel/results/trainLoss_Accuracy.npy')
validationLA = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesModel/results/validationLoss_Accuracy.npy')

In [ ]:
train_iterator = trange(0, 2, desc="Epoch")
optimizer = torch.optim.AdamW(model.parameters())
for _ in train_iterator:
    train_loss = 0.0
    nb_train_steps = 0
    preds = np.empty((0), dtype=np.int64)
    out_label_ids = np.empty((0), dtype=np.int64)
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")
    for step, (input_ids, labels) in enumerate(epoch_iterator):
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids.to("cuda"), labels=labels.to("cuda"))
        loss = outputs[0]
        loss.backward()
        logits = outputs[1]
        train_loss += loss.mean().item()
        nb_train_steps += 1
        preds = np.append(preds, logits.argmax(dim=1).detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, labels.detach().cpu().numpy(), axis=0)
        optimizer.step()
        del input_ids
        del labels
        del outputs
        torch.cuda.empty_cache()
    train_loss = train_loss / nb_train_steps
    acc = simple_accuracy(preds, out_label_ids)
    f1 = f1_score(y_true=out_label_ids, y_pred=preds, average="macro")
    print("=== Train: loss ===", train_loss)
    print("=== Train: acc. ===", acc)
    print("=== Train: f1 ===", f1)
    model.save_pretrained("/content/gdrive/MyDrive/MLA Project/10LanguagesModel/model")
    val = evaluate()
    trainLA = np.append(trainLA, [[train_loss, acc]],axis=0)
    validationLA = np.append(validationLA, [val],axis=0)
    np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesModel/results/trainLoss_Accuracy.npy', trainLA)
    np.save('/content/gdrive/MyDrive/MLA Project/10LanguagesModel/results/validationLoss_Accuracy.npy', validationLA)
    model.train()

Iteration: 100%|██████████| 1533/1533 [1:01:44<00:00,  2.42s/it]
/usr/local/lib/python3.7/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


=== Train: loss === 0.7205470244400811
=== Train: acc. === 0.7582817582103857
=== Train: f1 === 0.7598821172948759



Eval: 100%|██████████| 219/219 [08:26<00:00,  2.31s/it]
/usr/local/lib/python3.7/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Epoch:  50%|█████     | 1/2 [1:10:11<1:10:11, 4211.73s/it]

=== Eval: loss === 0.5416612482234223
=== Eval: acc. === 0.83755620583827
=== Eval: f1 === 0.8411714830192605



Iteration: 100%|██████████| 1533/1533 [1:01:54<00:00,  2.42s/it]
/usr/local/lib/python3.7/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


=== Train: loss === 0.7235844297223983
=== Train: acc. === 0.7578025428999664
=== Train: f1 === 0.7594172847415417



Eval: 100%|██████████| 219/219 [08:28<00:00,  2.32s/it]
/usr/local/lib/python3.7/dist-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Epoch: 100%|██████████| 2/2 [2:20:36<00:00, 4218.19s/it]

=== Eval: loss === 0.5059552247121454
=== Eval: acc. === 0.8410891442438084
=== Eval: f1 === 0.8450940006834597


**Final** **Test**

In [ ]:
#Using Test
_ = evaluate(True)

#**Plots**

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def Plotting(trainLA,validationLA):
    trainAcc = trainLA[:,1]
    validationAcc = validationLA[:,1]
    trainLoss = trainLA[:,0]
    validationLoss = validationLA[:,0]
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig2 = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(go.Scatter(y=validationLoss, name="Validation Loss"), secondary_y=False)
    fig.add_trace(go.Scatter(y=trainLoss, name="Training Loss"), secondary_y=False)
    fig2.add_trace(go.Scatter(y=validationAcc, name="Validation Accuracy"), secondary_y=False)
    fig2.add_trace(go.Scatter(y=trainAcc, name="Train Accuracy"), secondary_y=False)

    # Add figure title
    fig.update_layout(title_text="Loss of Model")
    fig2.update_layout(title_text="Accuracy of Model")

    # Set x-axis title
    fig.update_xaxes(title_text="Epoch")
    fig.update_yaxes(title_text="Loss")
    fig2.update_xaxes(title_text="Epoch")
    fig2.update_yaxes(title_text="Accuracy")
    fig.show()
    fig2.show()
    return

## 10 Programming Languages without comments

In [ ]:
trainLA = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesModel/results/trainLoss_Accuracy.npy')
validationLA = np.load('/content/gdrive/MyDrive/MLA Project/10LanguagesModel/results/validationLoss_Accuracy.npy')

In [ ]:
Plotting(trainLA,validationLA)

## 10 Programming Languages the whole transformer

In [ ]:
trainLA = np.load('/content/gdrive/MyDrive/MLA Project/training whole model/trainLoss_Accuracy.npy')
validationLA = np.load('/content/gdrive/MyDrive/MLA Project/training whole model/validationLoss_Accuracy.npy')

In [ ]:
Plotting(trainLA,validationLA)